<a href="https://colab.research.google.com/github/DilshanBotheju/CSACP_Project/blob/main/HomicidePatternRecognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
import numpy as np
import pandas as pd

In [27]:
#Load relevant datasets
homicide_data_path = "/content/drive/MyDrive/DSGP/datasets/database.csv"
homicide_data = pd.read_csv(homicide_data_path, encoding = "latin-1")
homicide_data.head()

<ipython-input-27-acfac49e8cac>:3: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  homicide_data = pd.read_csv(homicide_data_path, encoding = "latin-1")


,Record ID,Agency Code,Agency Name,Agency Type,City,State,Year,Month,Incident,Crime Type,...,Victim Ethnicity,Perpetrator Sex,Perpetrator Age,Perpetrator Race,Perpetrator Ethnicity,Relationship,Weapon,Victim Count,Perpetrator Count,Record Source
0,1,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,January,1,Murder or Manslaughter,...,Unknown,Male,15,Native American/Alaska Native,Unknown,Acquaintance,Blunt Object,0,0,FBI
1,2,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,March,1,Murder or Manslaughter,...,Unknown,Male,42,White,Unknown,Acquaintance,Strangulation,0,0,FBI
2,3,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,March,2,Murder or Manslaughter,...,Unknown,Unknown,0,Unknown,Unknown,Unknown,Unknown,0,0,FBI
3,4,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,April,1,Murder or Manslaughter,...,Unknown,Male,42,White,Unknown,Acquaintance,Strangulation,0,0,FBI
4,5,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,April,2,Murder or Manslaughter,...,Unknown,Unknown,0,Unknown,Unknown,Unknown,Unknown,0,1,FBI


In [28]:
# Identifying shape of the original dataset
homicide_data.shape

(638454, 24)

In [29]:
# Getting a sample percentage of data to preprocess
sample_percentage = 0.5
homicide_data = homicide_data.sample(frac=sample_percentage, random_state=42)

In [30]:
# Identifying shape of sample dataset
homicide_data.shape

(319227, 24)

In [31]:
# Analyisng crime dataset
homicide_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 319227 entries, 484984 to 190407
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   Record ID              319227 non-null  int64 
 1   Agency Code            319227 non-null  object
 2   Agency Name            319227 non-null  object
 3   Agency Type            319227 non-null  object
 4   City                   319227 non-null  object
 5   State                  319227 non-null  object
 6   Year                   319227 non-null  int64 
 7   Month                  319227 non-null  object
 8   Incident               319227 non-null  int64 
 9   Crime Type             319227 non-null  object
 10  Crime Solved           319227 non-null  object
 11  Victim Sex             319227 non-null  object
 12  Victim Age             319227 non-null  int64 
 13  Victim Race            319227 non-null  object
 14  Victim Ethnicity       319227 non-null  object
 15  

In [32]:
# Finding duplicates in dataset
homicide_data.duplicated().sum()

0

In [33]:
# Replacing unknown with NA to identify null columns
homicide_data.replace("Unknown", np.nan, inplace=True)

In [34]:
# Identifying null value count
homicide_data.isnull().sum()

,0
Record ID,0
Agency Code,0
Agency Name,26
Agency Type,0
City,0
State,0
Year,0
Month,0
Incident,0
Crime Type,0


In [35]:
homicide_data.head(20)

,Record ID,Agency Code,Agency Name,Agency Type,City,State,Year,Month,Incident,Crime Type,...,Victim Ethnicity,Perpetrator Sex,Perpetrator Age,Perpetrator Race,Perpetrator Ethnicity,Relationship,Weapon,Victim Count,Perpetrator Count,Record Source
484984,484985,CA03612,Victorville,Municipal Police,San Bernardino,California,2005,July,1,Murder or Manslaughter,...,Hispanic,NaN,0,NaN,NaN,NaN,Handgun,1,0,FBI
608445,608446,TX10100,Harris County,Sheriff,Harris,Texas,2013,May,3,Murder or Manslaughter,...,Not Hispanic,NaN,0,NaN,NaN,NaN,Firearm,0,0,FBI
417612,417613,AR06200,St. Francis,Sheriff,St. Francis,Arkansas,2001,June,1,Murder or Manslaughter,...,Not Hispanic,Male,18,Black,Not Hispanic,Acquaintance,Handgun,0,0,FBI
372692,372693,AZ01003,Tucson,Municipal Police,Pima,Arizona,1998,March,6,Murder or Manslaughter,...,Not Hispanic,NaN,0,NaN,NaN,NaN,Handgun,0,0,FBI
402028,402029,WA02400,Okanogan,Sheriff,Okanogan,Washington,1999,August,1,Murder or Manslaughter,...,NaN,Male,20,White,NaN,Girlfriend,Rifle,0,0,FBI
541091,541092,MI40502,Kalkaska,Municipal Police,Kalkaska,Michigan,2008,August,1,Murder or Manslaughter,...,NaN,Male,25,White,NaN,NaN,Blunt Object,0,2,FBI
432591,432592,WA03200,Spokane,Sheriff,Spokane,Washington,2001,March,1,Murder or Manslaughter,...,NaN,NaN,0,NaN,NaN,NaN,Handgun,0,0,FBI
489735,489736,MDBPD00,Baltimore,Municipal Police,Baltimore city,Maryland,2005,January,13,Murder or Manslaughter,...,NaN,NaN,0,NaN,NaN,NaN,Handgun,0,0,FBI
630057,630058,LA02800,Lafayette County,Sheriff,Lafayette,Louisiana,2014,February,1,Murder or Manslaughter,...,Not Hispanic,Male,15,Black,Not Hispanic,Husband,Firearm,0,0,FBI
558145,558146,NC03606,Gastonia,Municipal Police,Gaston,North Carolina,2009,April,1,Murder or Manslaughter,...,Not Hispanic,Male,21,Black,Not Hispanic,Stranger,Handgun,0,1,FBI


In [36]:
# Drop columns with higher null values
homicide_data = homicide_data.drop(["Perpetrator Ethnicity"],axis = 1,errors="ignore")

In [37]:
homicide_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 319227 entries, 484984 to 190407
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Record ID          319227 non-null  int64 
 1   Agency Code        319227 non-null  object
 2   Agency Name        319201 non-null  object
 3   Agency Type        319227 non-null  object
 4   City               319227 non-null  object
 5   State              319227 non-null  object
 6   Year               319227 non-null  int64 
 7   Month              319227 non-null  object
 8   Incident           319227 non-null  int64 
 9   Crime Type         319227 non-null  object
 10  Crime Solved       319227 non-null  object
 11  Victim Sex         318730 non-null  object
 12  Victim Age         319227 non-null  int64 
 13  Victim Race        315981 non-null  object
 14  Victim Ethnicity   134954 non-null  object
 15  Perpetrator Sex    224098 non-null  object
 16  Perpetrator Age    3

In [38]:
# Handling categorical null values using mode
for column in homicide_data.columns:
    if homicide_data[column].dtype == "object":
        homicide_data[column].fillna(homicide_data[column].mode()[0], inplace=True)

<ipython-input-38-83883757957c>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  homicide_data[column].fillna(homicide_data[column].mode()[0], inplace=True)


In [39]:
# Handling numerical null values using median
for column in homicide_data.columns:
    if homicide_data[column].dtype != "object":
        homicide_data[column].fillna(homicide_data[column].median(), inplace=True)

<ipython-input-39-a13c21e4658e>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  homicide_data[column].fillna(homicide_data[column].median(), inplace=True)


In [40]:
# Checking null count
homicide_data.isnull().sum()

,0
Record ID,0
Agency Code,0
Agency Name,0
Agency Type,0
City,0
State,0
Year,0
Month,0
Incident,0
Crime Type,0


In [41]:
# Viewing updated data
homicide_data.head(10)

,Record ID,Agency Code,Agency Name,Agency Type,City,State,Year,Month,Incident,Crime Type,...,Victim Race,Victim Ethnicity,Perpetrator Sex,Perpetrator Age,Perpetrator Race,Relationship,Weapon,Victim Count,Perpetrator Count,Record Source
484984,484985,CA03612,Victorville,Municipal Police,San Bernardino,California,2005,July,1,Murder or Manslaughter,...,White,Hispanic,Male,0,White,Acquaintance,Handgun,1,0,FBI
608445,608446,TX10100,Harris County,Sheriff,Harris,Texas,2013,May,3,Murder or Manslaughter,...,White,Not Hispanic,Male,0,White,Acquaintance,Firearm,0,0,FBI
417612,417613,AR06200,St. Francis,Sheriff,St. Francis,Arkansas,2001,June,1,Murder or Manslaughter,...,Black,Not Hispanic,Male,18,Black,Acquaintance,Handgun,0,0,FBI
372692,372693,AZ01003,Tucson,Municipal Police,Pima,Arizona,1998,March,6,Murder or Manslaughter,...,White,Not Hispanic,Male,0,White,Acquaintance,Handgun,0,0,FBI
402028,402029,WA02400,Okanogan,Sheriff,Okanogan,Washington,1999,August,1,Murder or Manslaughter,...,White,Not Hispanic,Male,20,White,Girlfriend,Rifle,0,0,FBI
541091,541092,MI40502,Kalkaska,Municipal Police,Kalkaska,Michigan,2008,August,1,Murder or Manslaughter,...,White,Not Hispanic,Male,25,White,Acquaintance,Blunt Object,0,2,FBI
432591,432592,WA03200,Spokane,Sheriff,Spokane,Washington,2001,March,1,Murder or Manslaughter,...,White,Not Hispanic,Male,0,White,Acquaintance,Handgun,0,0,FBI
489735,489736,MDBPD00,Baltimore,Municipal Police,Baltimore city,Maryland,2005,January,13,Murder or Manslaughter,...,Black,Not Hispanic,Male,0,White,Acquaintance,Handgun,0,0,FBI
630057,630058,LA02800,Lafayette County,Sheriff,Lafayette,Louisiana,2014,February,1,Murder or Manslaughter,...,White,Not Hispanic,Male,15,Black,Husband,Firearm,0,0,FBI
558145,558146,NC03606,Gastonia,Municipal Police,Gaston,North Carolina,2009,April,1,Murder or Manslaughter,...,Black,Not Hispanic,Male,21,Black,Stranger,Handgun,0,1,FBI


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

# Label encoding categorical features
categorical_cols = ["Agency Type", "City", "State", "Month", "Crime Type",
                    "Crime Solved", "Victim Sex", "Victim Race", "Victim Ethnicity",
                    "Perpetrator Sex", "Perpetrator Race", "Relationship", "Weapon"]
encoder_dict = {}

for col in categorical_cols:
    encoder = LabelEncoder()
    homicide_data[col] = encoder.fit_transform(homicide_data[col])
    encoder_dict[col] = encoder



In [ ]:
# Drop non-essential columns
homicide_data = homicide_data.drop(columns=["Record ID", "Incident", "Record Source"],axis=1,errors="ignore")



In [ ]:
# Cosine Similarity
similarity_matrix = cosine_similarity(homicide_data)
def find_similar_cases(case_index, top_n=5):
    similarities = similarity_matrix[case_index]
    similar_indices = np.argsort(similarities)[::-1][1:top_n+1]
    return  homicide_data.iloc[similar_indices]